In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install transformers==2.11.0
!pip install emoji

In [ ]:
import torch
import re
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import accuracy_score,matthews_corrcoef
import torch.nn.functional as F

from tqdm import tqdm, trange,tnrange,tqdm_notebook
import random
import os
import io
import re
import numpy as np
import os
from collections import Counter
import itertools
import emoji
import unicodedata
import matplotlib.pyplot as plt
import transformers
print(transformers.__version__)
print(torch.__version__)
#% matplotlib inline

In [ ]:
# identify and specify the GPU as the device, later in training loop we will load data into devic'''e
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

SEED = 19

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device == torch.device("cuda"):
    torch.cuda.manual_seed_all(SEED)
    
print(device)

BertTokenizer to run end-to-end tokenization: punctuation splitting + word piece. 
BertForSequenceClassification is the Bert Model transformer with a sequence classification/regression head on top (a linear layer on top of the pooled output). 
BertConfig is the configuration class to store model configurations. 
AdamW implements Adam learning rate optimization algorithm, it is a type of Stochastic Gradient Descent with momentum. Here momentum is described as the moving average of the gradient instead of gradient itself.
get_linear_schedule_with_warmup creates a schedule with a learning rate that decreases linearly after linearly increasing during a warm-up period.

In [ ]:
def load_dict_contractions():
    return {
        "u":"you",
        "doin":"doing",
        "jus":"just",
        "jst":"just",
        "waitin":"waiting",
        "whatchu":"what are you",
        "wtf":"what the fuck",
        "ugh": "disappointment",
        "omg": "oh my god", 
        "lol":"laugh out loud",
        "iv":"i have",
        "let's":"let us",
        "dont":"do not",
        "2":"to",
        "im": "i am",
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"can not",
        "cant":"can not",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }
global CONTRACTIONS
CONTRACTIONS = load_dict_contractions()
CONTRACTIONS = {k.lower():v.lower() for k, v in CONTRACTIONS.items()}
CONTRACTIONS1 = {}
for k, v in CONTRACTIONS.items():
  if "'" in k:
    CONTRACTIONS1[k.replace("'", '')] = v
  else:
    pass
CONTRACTIONS.update(CONTRACTIONS1)


def tweet_cleaning_for_sentiment_analysis(tweet):    
    tweet = tweet.replace("’","'")
    tweet = re.sub(r"http[^\s]+", " ", tweet)
    tweet = re.sub(r"https[^\s]+", " ", tweet)
    tweet = re.sub(r"RT @\S+", " ", tweet)
    tweet = re.sub(r"rt @\S+", " ", tweet)
    tweet = re.sub(r"rt", " ", tweet)
    tweet = re.sub(r"Rt", " ", tweet)
    tweet = re.sub(r"#\w+", " ", tweet)
    tweet = tweet.lower()
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    tweet = " ".join(reformed)
    
    #Deal with emojis
    tweet = emoji.demojize(tweet)
    #tweet = re.sub(r"http+", "", tweet)

    #tweet = tweet.replace(":"," ")
    #Removal of Punctuation
    #tweet = re.sub(r'[^a-zA-z\s\t]', '', tweet)    


    tweet = unicodedata.normalize('NFKD', tweet).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9']+", ' ', str(tweet).lower()).strip()
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    text = ' '.join(text.split())
    return text


def preprocess(text, stem=False):
    # Remove link,user and special characters
    #text = re.sub("@\S+|https?:\S+|http?:\S", ' ', str(text).lower()).strip()
    #text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    text = ' '.join(text.split())
    return text

In [ ]:
print(tweet_cleaning_for_sentiment_analysis("this is good 😀 the"))

In [ ]:
df = pd.read_csv('/kaggle/input/emotion-dataset-1/Semeval_2007.csv', encoding ="UTF-8")
df = df[['text', 'sentiments']]
print(df.shape)
df['text'] = df.text.apply(tweet_cleaning_for_sentiment_analysis)
df.text = df.text.apply(lambda x: preprocess(x))
df['len'] = df.text.apply(lambda x: len(x.split()))
df = df.dropna()
a = [True if i>2 else False for i in df['len'].values]
df = df[a]
a = [True if i<50 else False for i in df['len'].values]
df = df[a]
df1 = df
print(df.sentiments.value_counts())
fig, axes = plt.subplots(nrows=1, ncols=5)
for index, senti in enumerate(set(df.sentiments.values)):
    df[df['sentiments']==senti].groupby(['len']).size().plot(ax=axes[index], title=senti) 
plt.show()


df = pd.read_csv('/kaggle/input/emotion-dataset-1/emotion_dataset1_1.csv', encoding ="UTF-8")
df = df[['text', 'sentiments']]
print(df.shape)
df = df.dropna()
df['text'] = df.text.apply(tweet_cleaning_for_sentiment_analysis)
df.text = df.text.apply(lambda x: preprocess(x))
df['len'] = df.text.apply(lambda x: len(x.split()))
a = [True if i>2 else False for i in df['len'].values]
df = df[a]
a = [True if i<50 else False for i in df['len'].values]
df = df[a]
df2 = df
print(df.sentiments.value_counts())
fig, axes = plt.subplots(nrows=1, ncols=5)
for index, senti in enumerate(set(df.sentiments.values)):
    df[df['sentiments']==senti].groupby(['len']).size().plot(ax=axes[index], title=senti) 
plt.show()


df = pd.read_csv('/kaggle/input/emotion-dataset-1/tweet_clean.csv', encoding ="UTF-8")
df = df[['text', 'sentiments']]
print(df.columns)
df = df.dropna()
df = df[df['sentiments'] != 'disgust']
df['text'] = df.text.apply(tweet_cleaning_for_sentiment_analysis)
df.text = df.text.apply(lambda x: preprocess(x))
df['len'] = df.text.apply(lambda x: len(x.split()))
df = df.dropna()
a = [True if i>2 else False for i in df['len'].values]
df = df[a]
a = [True if i<50 else False for i in df['len'].values]
df = df[a]
df3 = df
print(df.sentiments.value_counts())
fig, axes = plt.subplots(nrows=1, ncols=5)
for index, senti in enumerate(set(df.sentiments.values)):
    df[df['sentiments']==senti].groupby(['len']).size().plot(ax=axes[index], title=senti) 
plt.show()


df = pd.read_csv('/kaggle/input/emotion-dataset-1/text_emotion.csv', encoding ="UTF-8", names = ['sentiments', '1', 'text'])[1:]
df = df[['text', 'sentiments']]
df = df.dropna()
df['text'] = df.text.apply(tweet_cleaning_for_sentiment_analysis)
df.text = df.text.apply(lambda x: preprocess(x))
df['len'] = df.text.apply(lambda x: len(x.split()))
delete = ["empty", "enthusiasm", "boredom", "anger", "neutral"]
a = [False if i in delete else True for i in df['sentiments'].values]
df = df[a]
df = df.dropna()
a = [True if i>2 else False for i in df['len'].values]
df = df[a]
a = [True if i<50 else False for i in df['len'].values]
df = df[a]
df4 = df
print(df.sentiments.value_counts())
fig, axes = plt.subplots(nrows=1, ncols=9)
for index, senti in enumerate(set(df.sentiments.values)):
    df[df['sentiments']==senti].groupby(['len']).size().plot(ax=axes[index], title=senti) 
plt.show()


df = pd.read_csv('/kaggle/input/emotion-dataset-1/Semeval_2007.csv', encoding ="UTF-8")
df = df[['text', 'sentiments']]
print(df.shape)
df = df.dropna()
df['text'] = df.text.apply(tweet_cleaning_for_sentiment_analysis)
df.text = df.text.apply(lambda x: preprocess(x))
df['len'] = df.text.apply(lambda x: len(x.split()))
df = df.dropna()
a = [True if i>2 else False for i in df['len'].values]
df = df[a]
a = [True if i<50 else False for i in df['len'].values]
df = df[a]
df5 = df
print(df.sentiments.value_counts())
fig, axes = plt.subplots(nrows=1, ncols=5)
for index, senti in enumerate(set(df.sentiments.values)):
    df[df['sentiments']==senti].groupby(['len']).size().plot(ax=axes[index], title=senti) 
plt.show()


df = pd.read_csv('/kaggle/input/emotion-dataset-1/Emotion Phrases.csv', encoding ="UTF-8", names=['sentiments', 'text'])
df = df[['text', 'sentiments']]
print(df.head(5))
df = df.dropna()
df['text'] = df.text.apply(tweet_cleaning_for_sentiment_analysis)
df.text = df.text.apply(lambda x: preprocess(x))
df['len'] = df.text.apply(lambda x: len(x.split()))
df = df.dropna()
a = [True if i>2 else False for i in df['len'].values]
df = df[a]
a = [True if i<50 else False for i in df['len'].values]
df = df[a]
df6 = df
print(df.sentiments.value_counts())
fig, axes = plt.subplots(nrows=1, ncols=7)
for index, senti in enumerate(set(df.sentiments.values)):
    df[df['sentiments']==senti].groupby(['len']).size().plot(ax=axes[index], title=senti) 
plt.show()


df = pd.read_csv('/kaggle/input/emotion-dataset-1/text-classification.csv', encoding ="UTF-8")
df = df[['text', 'sentiments']]
print(df.head(5))
df = df.dropna()
df['text'] = df.text.apply(tweet_cleaning_for_sentiment_analysis)
df.text = df.text.apply(lambda x: preprocess(x))
df['len'] = df.text.apply(lambda x: len(x.split()))
df = df.dropna()
a = [True if i>2 else False for i in df['len'].values]
df = df[a]
a = [True if i<50 else False for i in df['len'].values]
df = df[a]
df7 = df
print(df.sentiments.value_counts())
fig, axes = plt.subplots(nrows=1, ncols=5)
for index, senti in enumerate(set(df.sentiments.values)):
    df[df['sentiments']==senti].groupby(['len']).size().plot(ax=axes[index], title=senti) 
plt.show()


df = pd.read_csv('/kaggle/input/emotion-dataset-1/DATA.csv', encoding='iso-8859-1')
df = df[['text', 'sentiments']]
print(df.head(5))
df = df.dropna()
df['text'] = df.text.apply(tweet_cleaning_for_sentiment_analysis)
df.text = df.text.apply(lambda x: preprocess(x))
df['len'] = df.text.apply(lambda x: len(x.split()))
df = df.dropna()
a = [True if i>2 else False for i in df['len'].values]
df = df[a]
a = [True if i<50 else False for i in df['len'].values]
df = df[a]
df8 = df
print(df.sentiments.value_counts())
fig, axes = plt.subplots(nrows=1, ncols=7)
for index, senti in enumerate(set(df.sentiments.values)):
    df[df['sentiments']==senti].groupby(['len']).size().plot(ax=axes[index], title=senti) 
plt.show()

df = pd.read_csv('/kaggle/input/emotion-dataset-1/sadness_happiness_love_surprise.csv', encoding='iso-8859-1')
df = df[['text', 'sentiments']]
print(df.head(5))
df['text'] = df.text.apply(tweet_cleaning_for_sentiment_analysis)
df.text = df.text.apply(lambda x: preprocess(x))
print(df.head(5))
df['len'] = df.text.apply(lambda x: len(x.split()))
df = df.dropna()
a = [True if i>2 else False for i in df['len'].values]
df = df[a]
a = [True if i<50 else False for i in df['len'].values]
df = df[a]
df9 = df
print(df.sentiments.value_counts())
fig, axes = plt.subplots(nrows=1, ncols=4)
for index, senti in enumerate(set(df.sentiments.values)):
    df[df['sentiments']==senti].groupby(['len']).size().plot(ax=axes[index], title=senti) 
plt.show()


df = pd.read_csv('/kaggle/input/emotion-dataset-1/neutral.csv', encoding='iso-8859-1')
df = df[['text', 'sentiments']]
print(df.head(5))
df = df.dropna()
df['text'] = df.text.apply(tweet_cleaning_for_sentiment_analysis)
df.text = df.text.apply(lambda x: preprocess(x))
print(df.head(5))
df['len'] = df.text.apply(lambda x: len(x.split()))
df = df.dropna()
a = [True if i>2 else False for i in df['len'].values]
df = df[a]
a = [True if i<50 else False for i in df['len'].values]
df = df[a]
df10 = df
print(df.sentiments.value_counts())
fig, axes = plt.subplots(nrows=1, ncols=4)
for index, senti in enumerate(set(df.sentiments.values)):
    df[df['sentiments']==senti].groupby(['len']).size().plot(ax=axes[index], title=senti) 
plt.show()


df = pd.read_csv('/kaggle/input/emotion-dataset-1/Hashtag_data.csv', encoding='iso-8859-1')
df = df[['text', 'sentiments']]
print(df.head(5))
df = df.dropna()
df['text'] = df.text.apply(tweet_cleaning_for_sentiment_analysis)
df.text = df.text.apply(lambda x: preprocess(x))
print(df.head(5))
df['len'] = df.text.apply(lambda x: len(x.split()))
df = df.dropna()
a = [True if i>3 else False for i in df['len'].values]
df = df[a]
a = [True if i<50 else False for i in df['len'].values]
df = df[a]
df11 = df
print(df.sentiments.value_counts())
fig, axes = plt.subplots(nrows=1, ncols=6)
for index, senti in enumerate(set(df.sentiments.values)):
    df[df['sentiments']==senti].groupby(['len']).size().plot(ax=axes[index], title=senti) 
plt.show()

#df_h1 = df[df['sentiments'] != 'happy'][:5000]
#df_h2 = df[df['sentiments'] == 'sadness'][:5000]
df_h3 = df[df['sentiments'] == 'anger'][:5000]
df_h4 = df[df['sentiments'] == 'fear'][:5000]
df_h5 = df[df['sentiments'] == 'love'][:5000]
#df_h6 = df[df['sentiments'] == 'surprise'][:5000]
df_h = pd.concat([df_h3, df_h4, df_h5])


df = pd.read_csv('/kaggle/input/emotion-dataset-1/tested_correct.csv', encoding='iso-8859-1')
df = df[['text', 'sentiments']]
df = df.dropna()
df['text'] = df.text.apply(tweet_cleaning_for_sentiment_analysis)
df.text = df.text.apply(lambda x: preprocess(x))
print(df.head(5))
df['len'] = df.text.apply(lambda x: len(x.split()))
df = df.dropna()
df7_ = df

In [ ]:
df = pd.concat([ df2, df3, df4, df5, df6, df7, df8, df9, df10, df_h])
print(df.sentiments.value_counts())
df = df.drop_duplicates(subset=["text"])
df.loc[df.sentiments == "sad", "sentiments"] = "sadness"
df.loc[(df.sentiments == "joy")|(df.sentiments == "Happy"), "sentiments"] = "happy"
df = df[df['sentiments'] != 'worry']
df.loc[df.sentiments == "happiness", "sentiments"] = "happy"
df = df[(df['sentiments'] != 'disgust')&(df['sentiments'] != 'shame')&(df['sentiments'] != 'guilt')]
df.loc[(df.sentiments == "fun")|(df.sentiments == "relief"), "sentiments"] = "happy"
df.loc[df.sentiments == "hate", "sentiments"] = "anger"
#df.loc[df.sentiments == "surprise", "sentiments"] = "love"
#df.loc[df.sentiments == "love", "sentiments"] = "excited"
print(df.head(5))
print(df.sentiments.value_counts())


In [ ]:
'''number_category = 7000
df1_ = df[df['sentiments']=='happy']
df2_ = df[df['sentiments']=='sadness']
df3_ = df[df['sentiments']=='excited']
df4_ = df[df['sentiments']=='anger']
df5_ = df[df['sentiments']=='fear']
df6_ = df[df['sentiments']=='neutral']
df7_ = df[df['sentiments']=='love']
df_ = pd.concat([df1_, df2_, df3_, df4_, df5_, df6_, df7_])
df_.to_csv('a.csv')'''

In [ ]:
for i in set(df.sentiments.values):
    print(i)

In [ ]:
print(df.sentiments.value_counts())

In [ ]:

number_category = 7000
df1_ = df[df['sentiments']=='happy'][:]
df2_ = df[df['sentiments']=='sadness'][:10000]
df3_ = df[df['sentiments']=='love']
df4_ = df[df['sentiments']=='anger'][:10000]
df5_ = df[df['sentiments']=='fear'][:10000]
df6_ = df[df['sentiments']=='neutral']
df_ = pd.concat([df1_, df2_, df3_, df4_, df5_, df6_, df7_])
df_.loc[(df_.sentiments == "happy")|(df_.sentiments == "love"), "sentiments"] = "positive"
df_.loc[(df_.sentiments == "sadness")|(df_.sentiments == "anger")|(df_.sentiments == "fear"), "sentiments"] = "negative"
df_ = df_[df_['sentiments']!='excited']
df_ = df_[df_['sentiments']!='positive']
df_ = df_[df_['sentiments']!='negative']

df_neg_pos = pd.read_csv('/kaggle/input/emotion-dataset-1/Sentiment_50000.csv')
df_neg_pos = df_neg_pos[['sentiments', 'text']]
df_neg_pos.loc[(df_neg_pos.sentiments == 0), "sentiments"] = "negative"
df_neg_pos.loc[(df_neg_pos.sentiments == 1), "sentiments"] = "positive"


df_BTC = pd.read_csv('/kaggle/input/emotion-dataset-1/BTC_tweets.csv')
df_BTC = df_BTC[['sentiments', 'text']]

df_1 = pd.concat([df_, df_neg_pos, df_BTC])
df_1['text'] = df_1.text.apply(tweet_cleaning_for_sentiment_analysis)
df_1.text = df_1.text.apply(lambda x: preprocess(x))
df_1 = df_1[['sentiments', 'text']]
df_1 = df_1.dropna()
df_1 = df_1.drop_duplicates(subset=["text"])
print(df_1.sentiments.value_counts())

In [ ]:
df_1 = pd.concat([df_, df_neg_pos, df_BTC])
df_1['text'] = df_1.text.apply(tweet_cleaning_for_sentiment_analysis)
df_1.text = df_1.text.apply(lambda x: preprocess(x))
print(df_1.sentiments.value_counts())

In [ ]:
df_1 = df_1[['sentiments', 'text']]
df_1 = df_1.dropna()
#df_1 = df_1.drop_duplicates(subset=["text"])
print(df_1.sentiments.value_counts())

In [ ]:
df = df_1
print(df_1.sentiments.value_counts())

In [ ]:
df

In [ ]:
df.rename(columns={'sentiments':'label'},inplace=True)
df.rename(columns={'text':'sentence'},inplace=True)
df.to_csv('a-22-11-20.csv')
df['label'].unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['label_enc'] = labelencoder.fit_transform(df['label'])

In [ ]:
df[['label','label_enc']].drop_duplicates(keep='first')

In [ ]:
df.label.value_counts()

In [ ]:
df.rename(columns={'label':'label_desc'},inplace=True)
df.rename(columns={'label_enc':'label'},inplace=True)

In [ ]:
## create label and sentence list
sentences = df.sentence.values

#check distribution of data based on labels
print("Distribution of data based on labels: ",df.label.value_counts())

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 256

## Import BERT tokenizer, that is used to convert our text into tokens that corresponds to BERT library
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
input_ids = [tokenizer.encode(sent, add_special_tokens=True,max_length=MAX_LEN,pad_to_max_length=True) for sent in sentences]
labels = df.label.values

print("Actual sentence before tokenization: ",sentences[2])
print("Encoded Input from dataset: ",input_ids[2])

## Create attention mask
attention_masks = []
## Create a mask of 1 for all input tokens and 0 for all padding tokens
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]
print(attention_masks[2])

## Dataset Prep for training

#### Split into a training set and a test set using a stratified k fold

In [ ]:
train_inputs,validation_inputs,train_labels,validation_labels = train_test_split(input_ids,labels,random_state=41,test_size=0.1)
train_masks,validation_masks,_,_ = train_test_split(attention_masks,input_ids,random_state=41,test_size=0.1)

In [ ]:
# convert all our data into torch tensors, required data type for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory
train_data = TensorDataset(train_inputs,train_masks,train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

validation_data = TensorDataset(validation_inputs,validation_masks,validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)

In [ ]:
train_data[0]

In [ ]:
type(train_dataloader)

In [ ]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3).to(device)

# Parameters:
lr = 2e-5
adam_epsilon = 1e-8

# Number of training epochs (authors recommend between 2 and 4)
epochs =3

num_warmup_steps = 0
num_training_steps = len(train_dataloader)*epochs

### In Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr,eps=adam_epsilon,correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

In [ ]:
## Store our loss and accuracy for plotting
train_loss_set = []
learning_rate = []
y_text_pred = []
y_text_orig = []

# Gradients gets accumulated by default
model.zero_grad()

# tnrange is a tqdm wrapper around the normal python range
for _ in tnrange(1,epochs+1,desc='Epoch'):
  print("<" + "="*22 + F" Epoch {_} "+ "="*22 + ">")
  # Calculate total loss for this epoch
  batch_loss = 0

  for step, batch in enumerate(train_dataloader):
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()
    
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    
    # Backward pass
    loss.backward()
    
    # Clip the norm of the gradients to 1.0
    # Gradient clipping is not in AdamW anymore
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    # Update learning rate schedule
    scheduler.step()

    # Clear the previous accumulated gradients
    optimizer.zero_grad()
    
    # Update tracking variables
    batch_loss += loss.item()

  # Calculate the average loss over the training data.
  avg_train_loss = batch_loss / len(train_dataloader)

  #store the current learning rate
  for param_group in optimizer.param_groups:
    print("\n\tCurrent Learning rate: ",param_group['lr'])
    learning_rate.append(param_group['lr'])
    
  train_loss_set.append(avg_train_loss)
  print(F'\n\tAverage Training loss: {avg_train_loss}')
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_accuracy,eval_mcc_accuracy,nb_eval_steps = 0, 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits[0].to('cpu').numpy()
    label_ids = b_labels.to('cpu').numpy()

    pred_flat = np.argmax(logits, axis=1).flatten()
    labels_flat = label_ids.flatten()
    y_text_orig = y_text_orig + list(labels_flat)
    y_text_pred = y_text_pred + list(pred_flat)
    
    df_metrics=pd.DataFrame({'Epoch':epochs,'Actual_class':labels_flat,'Predicted_class':pred_flat})
    
    tmp_eval_accuracy = accuracy_score(labels_flat,pred_flat)
    tmp_eval_mcc_accuracy = matthews_corrcoef(labels_flat, pred_flat)
    
    eval_accuracy += tmp_eval_accuracy
    eval_mcc_accuracy += tmp_eval_mcc_accuracy
    nb_eval_steps += 1

  print(F'\n\tValidation Accuracy: {eval_accuracy/nb_eval_steps}')
  print(F'\n\tValidation MCC Accuracy: {eval_mcc_accuracy/nb_eval_steps}')

In [ ]:
df[['label','label_desc']].drop_duplicates(keep='first')

In [ ]:
## emotion labels

label2int = { "negative": 0, "neutral": 1, "positive": 2}

In [ ]:
 df_metrics['Predicted_class'].unique()

In [ ]:
print(classification_report(df_metrics['Actual_class'].values, df_metrics['Predicted_class'].values, target_names=label2int.keys(), digits=len(label2int)))

In [ ]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

## emotion labels
#{ "anger": 0, "fear": 1, "happy": 2, "love":3, "neutral": 4, "sadness": 5}
label2int = { "negative": 0, "neutral": 1, "positive": 2}
int2label = { 0: "negative", 1:"neutral", 2:"positive"}
#y_text_pred = df_metrics['Predicted_class'].values
#y_text_orig = df_metrics['Actual_class'].values

#y_text_pred = [int2label[i] for i in y_text_pred]
#y_text_orig = [int2label[i] for i in y_text_orig]
#print((y_text_orig))
#print((y_text_pred))

In [ ]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=22)
    plt.yticks(tick_marks, classes, fontsize=22)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=25)
    plt.xlabel('Predicted label', fontsize=25)

cnf_matrix = confusion_matrix(y_text_orig, y_text_pred)
plt.figure(figsize=(12,12))
#{ "anger": 0, "fear": 1, "happy": 2, "love":3, "neutral": 4, "sadness": 5}
plot_confusion_matrix(cnf_matrix, classes=["negative", "neutral", "positive"], title="Confusion matrix")
plt.show()

In [ ]:
## emotion labels
labe = {
  "negative": 0,
  "neutral": 1,
  "positive": 2
}

In [ ]:
model = torch.load('/kaggle/input/sentiment-model-bin/Sentiment_model-21-1-16.bin', map_location=device).to(device)
tokenizer = torch.load('/kaggle/input/sentiment-model-bin/tokenizer-21-1-16.bin', map_location=device)

In [ ]:
def Emotion_model(sentences, model, tokenizer):
    label = ["negative", "neutral", "positive"]
    model1 = model.eval()
    input_ids = [tokenizer.encode(sent, add_special_tokens=True,max_length=256,pad_to_max_length=True) for sent in sentences]
    input_masks = [[float(i>0) for i in seq] for seq in list(input_ids)]
    #input_masks = input_masks.reshape([1,input_mask.shape])
    b_input_ids = torch.tensor(input_ids).to(device)
    b_input_mask = torch.tensor(input_masks).to(device)
    with torch.no_grad():
        outputs = model1(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    #print(F.softmax(outputs[0]).to('cpu').numpy()[0])
    print(['{:f}'.format(item) for item in F.softmax(outputs[0], dim=1).to('cpu').numpy()[0]*100])
    logits = outputs[0].to('cpu').numpy()
    pred_flat = np.argmax(logits, axis=1).flatten()
    return label[pred_flat[0]]

In [ ]:
def tweet_cleaning_for_sentiment_analysis1(tweet):    
    tweet = tweet.replace("’","'")
    tweet = re.sub(r"http[^\s]+", " ", tweet)
    tweet = re.sub(r"https[^\s]+", " ", tweet)
    #tweet = re.sub(r"[.,!]", " ", tweet)
    tweet = tweet.lower()
    words = tweet.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    tweet = " ".join(reformed)
    
    #Deal with emojis
    tweet = emoji.demojize(tweet)  

    tweet = unicodedata.normalize('NFKD', tweet).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub("https?:\S+|http?:\S|[^A-Za-z0-9@#']+", ' ', str(tweet).lower()).strip()
    text = ''.join(''.join(s)[:2] for _, s in itertools.groupby(text))
    text = ' '.join(text.split())
    return text

In [ ]:
sentences = ["""i am thrilled by you. """]
sentences = [tweet_cleaning_for_sentiment_analysis1(sentences[0])]
print(["  Negative ", "  Neutral ", "   Positive  "])
print(Emotion_model(sentences, model, tokenizer))

In [ ]:
torch.save(model,'Sentiment_model50000-22-12-09.bin')
torch.save(tokenizer,'tokenizer50000-22-12-09.bin')
df.to_csv('a50000-22-12-09.csv')
#df.to_csv('a-21-1-16.csv')

In [ ]:
df.to_csv('a-22-10-23.csv')

In [ ]:
'''
label = ["anger", "fear", "happy" , "love", "neutral", "sadness"]
df = pd.read_csv('/kaggle/input/emotion-analysis/Emotion  Sentiment analysis - Emotions.csv', encoding='iso-8859-1')
predict_neu = []
for sentences in df.Statements.values:
    sentences = [tweet_cleaning_for_sentiment_analysis1(sentences)]
    model = model.eval()
    predict_neu.append(Emotion_model(sentences, model, tokenizer))
df['predict'] = predict_neu
df.to_csv('predict_neu_lov_20_10.csv')
'''